In [1]:
import pandas as pd
import re

df = pd.read_csv('./evolucion_migracion.csv', sep=";")

# 2) Convertir "Total" (viene con separador de miles con puntos, ej: 49.128.297)
df["Total"] = (
    df["Total"].astype(str)
    .str.replace(".", "", regex=False)
    .str.replace(" ", "", regex=False)
)
df["Total"] = pd.to_numeric(df["Total"], errors="coerce")

# 3) Parsear "Periodo" (formato tipo: "1 de enero de 2025")
MONTHS = {
    "enero": 1, "febrero": 2, "marzo": 3, "abril": 4, "mayo": 5, "junio": 6,
    "julio": 7, "agosto": 8, "septiembre": 9, "setiembre": 9, "octubre": 10,
    "noviembre": 11, "diciembre": 12
}

def parse_periodo(s: str):
    s = str(s).strip().lower()
    m = re.match(r"(\d{1,2})\s+de\s+([a-zA-Zñ]+)\s+de\s+(\d{4})", s)
    if not m:
        return pd.NaT
    day = int(m.group(1))
    mon = MONTHS.get(m.group(2))
    year = int(m.group(3))
    return pd.Timestamp(year, mon, day) if mon else pd.NaT

df["Periodo_dt"] = df["Periodo"].apply(parse_periodo)

# 4) Filtrar último periodo + "Todas las edades" + "Total"
latest = df["Periodo_dt"].max()
base = df[
    (df["Periodo_dt"] == latest) &
    (df["Grupo quinquenal de edad"].str.strip() == "Todas las edades") &
    (df["Sexo"].str.strip() == "Total")
].copy()

In [2]:
base.head()

,Nacionalidad,Grupo quinquenal de edad,Sexo,Periodo,Total,Periodo_dt
0,Total,Todas las edades,Total,1 de enero de 2025,49128297.0,2025-01-01
5580,Española,Todas las edades,Total,1 de enero de 2025,42216326.0,2025-01-01
11160,País de la UE27_2020 sin España,Todas las edades,Total,1 de enero de 2025,1757955.0,2025-01-01
16740,País de la UE28 sin España,Todas las edades,Total,1 de enero de 2025,NaN,2025-01-01
22320,Bélgica,Todas las edades,Total,1 de enero de 2025,44644.0,2025-01-01


In [8]:
# 5) (Opcional pero recomendado) quitar agregados para quedarte con países
EXCLUDE_EXACT = {"Total", "Extranjera", "Apátrida", "No consta"}
EXCLUDE_PREFIXES = ("De ", "País de ", "Países de ", "Resto de ", "Otros", "Sin ")

total_population = (
    base.loc[base["Nacionalidad"].str.strip() == "Total", "Total"]
    .iloc[0]
)

mask = ~base["Nacionalidad"].str.strip().isin(EXCLUDE_EXACT)
mask &= ~base["Nacionalidad"].str.strip().str.startswith(EXCLUDE_PREFIXES)
base_countries = base[mask].copy()

# 7) Calcular porcentaje
base_countries["pct_total"] = (base_countries["Total"] / total_population) * 100

# 8) Top 5 porcentajes
top5_pct = (
    base_countries
    .sort_values("pct_total", ascending=False)
    .head(5)[["Nacionalidad", "Total", "pct_total"]]
    .reset_index(drop=True)
)

top5_pct["pct_total"] = top5_pct["pct_total"].round(3)

print(f"Último periodo: {latest.date()}")
print(top5_pct)

Último periodo: 2025-01-01
  Nacionalidad       Total  pct_total
0     Española  42216326.0     85.931
1    Marruecos    968999.0      1.972
2     Colombia    676534.0      1.377
3      Rumanía    609270.0      1.240
4    Venezuela    377809.0      0.769
